# Newell-Lee Evaporator: Comparison of SysID Models

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Libraries for Pyomo
from pyomo.environ import *
from pyomo.dae import DerivativeVar, ContinuousSet

# Libraries for MKCVA and CVA
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
from matplotlib import cm, colors
from scipy.linalg import cholesky
import matplotlib.pyplot as plt
import statsmodels.api as sm
from time import time
import pandas as pd
import numpy as np
import cyipopt
import pickle

# Libraries for LSTM
import gc
import keras
import tensorflow as tf
from keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.models import Model,load_model

## Declare all classes

In [2]:
class CVA:
    def __init__(self, verbose=None):
        if verbose == None:
            self.verbose = 0
        else:
            self.verbose = verbose
    
    def identify(self, Z_train, UI, YI, n_states=None):
        
        start = time()
        self.UI = UI  # Column indices of input vars 
        self.YI = YI  # Column indices of output vars
        N = Z_train.shape[0]

        # Perform Standard Scaling on raw data [u y]
        sc_raw = StandardScaler()
        Z_train_sc = sc_raw.fit_transform(Z_train)
        y_train = Z_train_sc[:, YI]
        self.sc_raw = sc_raw

        # Calculate the suggested no. of lags on the KPCA scores
        _, ci = sm.tsa.acf(np.sum(y_train ** 2, axis=1), alpha=0.05)
        self.n_lags = np.argwhere(ci[:,0] < 0)[0][0]
        p = f = self.n_lags
        
        # Create Hankel matrices: Yp and Yf from KPCA scores
        Yp, Yf = [], []
        for k in np.arange(N-p-f):
            Yp.append(np.flip(Z_train_sc[k:k+p, :].reshape(-1, 1)))

        for k in np.arange(1, N-p-f+1):
            Yf.append(Z_train_sc[k+p:k+p+f, YI].reshape(-1, 1))

        Yp = np.transpose(np.hstack(Yp))
        Yf = np.transpose(np.hstack(Yf))
        Np = Yp.shape[0]

        # Standardize the Hankel matrices
        sc_p = StandardScaler()
        sc_f = StandardScaler()
        Yp_scaled = sc_p.fit_transform(Yp)
        Yf_scaled = sc_f.fit_transform(Yf)
        self.sc_p = sc_p

        # Perform CCA
        Epp = cholesky(np.dot(Yp_scaled.T, Yp_scaled))  # Past Cholesky matrix
        Eff = cholesky(np.dot(Yf_scaled.T, Yf_scaled))  # Future Cholesky matrix
        Efp = np.dot(Yf_scaled.T, Yp_scaled)            # Cross-covariance matrix
        H = np.linalg.inv(Eff.T) @ Efp @ np.linalg.inv(Epp)

        U, S, V = np.linalg.svd(H)

        # Calculate the suggested no. of states via knee of SV plot
        if n_states == None:
            n_states = np.minimum(2 + np.argmax(np.diff(np.diff(S))), 10)
        self.n_states = n_states

        if self.verbose:
            print(f'No. of lags: {self.n_lags}')
            plt.plot(np.arange(15)+1, S[:15], 'b.--')
            plt.scatter(n_states, S[n_states-1], c='r')
            plt.title('Singular Values plot')
            print(f'No. of states: {n_states}')
            plt.grid()
            plt.tight_layout()
            plt.show()

        # Calculate the state vectors, X
        Vn = np.transpose(V[:n_states, :])
        Jn = np.dot(Vn.T, np.linalg.inv(Epp.T))
        X = Jn @ Yp_scaled.T
        self.Jn = Jn
        
        # Solve for A, B, C, D, K
        M = X.shape[1]
        tk = np.transpose(Z_train_sc[p-1:p+M-1, self.UI])
        yk = np.transpose(y_train[p:p+M, :])

        CD = yk[:,:(M-1)] @ np.linalg.pinv(np.vstack((X[:,:(M-1)], 
                                                      tk[:,:(M-1)])))
        C = CD[:len(YI), :n_states]          # Output matrix
        D = CD[:len(YI), n_states:]          # Feedthrough matrix

        E = yk[:,:(M-1)] - C @ X[:,:(M-1)] - D @ tk[:,:(M-1)]
        ABK = X[:,1:M] @ np.linalg.pinv(np.vstack((X[:,:(M-1)], 
                                                   tk[:,:(M-1)], E)))
        A = ABK[:,:n_states]                 # State transition matrix
        B = ABK[:,n_states:(n_states+len(UI))]  # Input matrix
        K = ABK[:,(n_states+len(UI)):]          # Kalman gain
        
        self.ident_time = time() - start     # Time elapsed for identification
        
        self.A, self.B, self.C, self.D, self.K = A, B, C, D, K
        self.X = X
       
    def init_sim(self, Z_test_sc):
        # Calculate initial state x(0) using CVA projection matrix
        yp = np.flip(Z_test_sc[:self.n_lags, :].reshape(1, -1))
        Yp_scaled = self.sc_p.transform(yp)
        x0 = self.Jn @ Yp_scaled.T
        return x0
    
    def simulate(self, Z_test):
        start = time()
        Nt = Z_test.shape[0]
        Z_test_sc = self.sc_raw.transform(Z_test)
        x_pred = np.zeros((self.n_states, Nt - self.n_lags + 1)) 
        y_pred = np.zeros((len(self.YI), Nt - self.n_lags + 1))
        x0 = self.init_sim(Z_test_sc)
        u0 = Z_test_sc[0, self.UI].reshape(-1, 1)
        y0 = self.C @ x0 + self.D @ u0
        x_pred[:, 0] = x0.ravel()
        y_pred[:, 0] = y0.ravel()
        exit_code = 0
        
        for j in np.arange(1, y_pred.shape[1]):
            uk = Z_test_sc[j+self.n_lags-1, self.UI].reshape(-1, 1)
            xk_1 = x_pred[:, j-1].reshape(-1, 1)
            xk = self.A @ xk_1 + self.B @ uk
            yk = self.C @ xk + self.D @ uk
            x_pred[:, j] = xk.ravel()
            y_pred[:, j] = yk.ravel()
            if (np.abs(yk) > 1e3).any():
                exit_code = -1
        
        x_pred = np.transpose(x_pred)
        y_pred = np.transpose(y_pred)
        y_pred = (y_pred * self.sc_raw.scale_[self.YI]) + \
                           self.sc_raw.mean_[self.YI]
        self.sim_time = time() - start
        
        return x_pred, y_pred, exit_code
    
    def R2_score(self, Z_test, y_pred):
        r2 = np.zeros(len(self.YI))
        for k in range(len(self.YI)):
            y_true = Z_test[self.n_lags-1:, self.YI[k]]
            r2[k] = 1 - np.sum((y_true - y_pred[:, k]) ** 2) \
                    / np.sum((y_true - np.mean(y_true)) ** 2)   
        return r2
    
    def display(self):
        print(f'No. of lags: {self.n_lags}')
        print(f'No. of states: {self.n_states}')
        print(f'Indices of u: {self.UI}')
        print(f'Indices of y: {self.YI}')
        print('State-space matrices:')
        print(self.A)
        print(self.B)
        print(self.C)
        print(self.D)
        print(self.K)
        
class KPCA:
    def kernel_func(self, x1, x2):
        # x1 size: [no. of samples x no. of features]
        # x2 size: [no. of samples x no. of features]
        
        D = np.sum((x1 / self.kw) ** 2, axis=1, keepdims=True) \
            + np.sum((x2 / self.kw).T ** 2, axis=0, keepdims=True) \
            - 2 * np.tensordot(x1 / self.kw**2, x2.T, axes=1)
        L = np.tensordot(x1, x2.T, axes=1) + 1
        return self.w * L + (1 - self.w) * np.exp(-D)
        
    def fit_transform(self, X, kw, w, n_comp=None):
        # Compute kernel matrix
        self.X = X
        self.kw = kw
        self.w = w
        self.N = len(self.X)
        
        # Calculate reduced kernel matrix with k-medoids clustering 
        dist = pairwise_distances(self.X)
        dm = kmedoids.KMedoids(method='fasterpam', 
                               n_clusters=int(0.2*self.N), 
                               random_state=0).fit(dist)
        self.X_red = X[dm.medoid_indices_, :]
        self.N_red = len(dm.medoid_indices_)
        self.medoid_indices = dm.medoid_indices_
        self.K = self.kernel_func(self.X_red, self.X_red)

        # Center the kernel matrix
        self.U = np.ones((self.N_red, self.N_red)) / self.N_red
        Kc = self.K - self.U @ self.K - self.K @ self.U + self.U @ self.K @ self.U

        # Perform eigenvalue decomposition
        eigvals, eigvecs = np.linalg.eigh(Kc / self.N_red)

        # Ensure the eigenvalues are sorted in decreasing order
        ind = (-eigvals).argsort()
        eigvals = eigvals[ind]
        eigvecs = eigvecs[:,ind]
        
        if n_comp == None:
            # Get eigenvalues using CPV = 99%
            CPV = np.cumsum(eigvals) / np.sum(eigvals)
            self.n_comp = np.argwhere(CPV > 0.99)[0][0]
        else:
            self.n_comp = n_comp
        
        self.CPV = CPV
        
        # Compute the projection matrix
        self.P = eigvecs[:,:self.n_comp] @ np.diag(eigvals[:self.n_comp] ** -0.5)
        
        # Project the training data X via the reduced centered kernel matrix
        scores = self.transform(self.X)
        return scores
    
    def transform(self, Xt):
        if len(Xt.shape) == 1:
            Xt = Xt.reshape(1, -1)
        
        Kt = np.transpose(self.kernel_func(self.X_red, Xt))
        Ut = np.ones((Xt.shape[0], self.N_red)) / self.N_red
        Kct = Kt - Ut @ self.K - Kt @ self.U + Ut @ self.K @ self.U
        scores = Kct @ self.P
        return scores
    
class MKCVA:
    def __init__(self, verbose=None):
        if verbose == None:
            self.verbose = 0
        else:
            self.verbose = verbose
    
    def identify(self, Z_train, UI, YI, kw=None, w=None, n_states=None):
        
        start = time()
        self.UI = UI  # Column indices of input vars 
        self.YI = YI  # Column indices of output vars
        N = Z_train.shape[0]

        # Perform Standard Scaling on raw data [u y]
        sc_raw = StandardScaler()
        Z_train_sc = sc_raw.fit_transform(Z_train)
        y_train = Z_train_sc[:, YI]
        self.sc_raw = sc_raw

        # Perform KPCA on scaled [u(k), y(k-1)] data
        
        Z_kpca = np.hstack((Z_train_sc[1:, self.UI], 
                            Z_train_sc[:-1, self.YI]))
        
        kpca_uy = KPCA()
        kpca_score_uy = kpca_uy.fit_transform(Z_kpca, kw, w)
        self.kpca_uy = kpca_uy
        
        # Perform KPCA on scaled [y] data
        kpca_y = KPCA()
        kw = kpca_uy.kw * (len(YI) / Z_train.shape[1])
        kpca_score_y = kpca_y.fit_transform(Z_train_sc[:,YI], kw[YI], w)
        N_uy, N_y = kpca_uy.n_comp, kpca_y.n_comp

        # Calculate the suggested no. of lags on the KPCA scores
        _, ci = sm.tsa.acf(np.sum(kpca_score_y ** 2, axis=1), alpha=0.05)
        self.n_lags = np.argwhere(ci[:,0] < 0)[0][0]
        p = f = self.n_lags
        
        # Create Hankel matrices: Yp and Yf from KPCA scores
        Yp, Yf, Yp_all = [], [], []
        for k in np.arange(N-p-f):
            Yp.append(np.flip(kpca_score_uy[k:k+p, :].reshape(-1, 1)))

        for k in np.arange(1, N-p-f+1):
            Yf.append(kpca_score_y[k+p:k+p+f, :].reshape(-1, 1))

        Yp = np.transpose(np.hstack(Yp))
        Yf = np.transpose(np.hstack(Yf))
        Np = Yp.shape[0]

        # Standardize the Hankel matrices
        sc_p = StandardScaler()
        sc_f = StandardScaler()
        Yp_scaled = sc_p.fit_transform(Yp)
        Yf_scaled = sc_f.fit_transform(Yf)
        self.sc_p = sc_p

        # Perform CCA
        Epp = cholesky(np.dot(Yp_scaled.T, Yp_scaled))  # Past Cholesky matrix
        Eff = cholesky(np.dot(Yf_scaled.T, Yf_scaled))  # Future Cholesky matrix
        Efp = np.dot(Yf_scaled.T, Yp_scaled)            # Cross-covariance matrix
        H = np.linalg.inv(Eff.T) @ Efp @ np.linalg.inv(Epp)

        U, S, V = np.linalg.svd(H)

        # Calculate the suggested no. of states via knee of SV plot
        if n_states == None:
            n_states = np.minimum(2 + np.argmax(np.diff(np.diff(S))), 10)
        self.n_states = n_states

        if self.verbose:
            print(f'No. of KPCs on [u y]: {N_uy}')
            print(f'No. of KPCs on [y]:   {N_y}')
            print(f'No. of lags: {self.n_lags}')
            plt.figure(figsize=(12, 3))
            plt.subplot(131)
            plt.title('CPV Plot for KPCA on [u y]')
            plt.plot(np.arange(len(kpca_uy.CPV))+1, kpca_uy.CPV, 'b.--')
            plt.scatter(kpca_uy.n_comp, kpca_uy.CPV[kpca_uy.n_comp-1], c='r')
            plt.xlim([0, 30])
            plt.grid()
            plt.subplot(132)
            plt.title('CPV Plot for KPCA on [y]')
            plt.plot(np.arange(len(kpca_y.CPV))+1, kpca_y.CPV, 'b.--')
            plt.scatter(kpca_y.n_comp, kpca_y.CPV[kpca_y.n_comp-1], c='r')
            plt.xlim([0, 30])
            plt.grid()
            plt.subplot(133)
            plt.plot(np.arange(15)+1, S[:15], 'b.--')
            plt.scatter(n_states, S[n_states-1], c='r')
            plt.title('Singular Values plot')
            print(f'No. of states: {n_states}')
            plt.grid()
            plt.tight_layout()
            plt.show()

        # Calculate the state vectors, X
        Vn = np.transpose(V[:n_states, :])
        Jn = np.dot(Vn.T, np.linalg.inv(Epp.T))
        X = Jn @ Yp_scaled.T
        self.Jn = Jn
        
        # Solve for A, B, C, D, K
        M = X.shape[1]
        tk = np.transpose(kpca_score_uy[p-1:p+M-1, :])
        yk = np.transpose(y_train[p:p+M, :])

        CD = yk[:,:(M-1)] @ np.linalg.pinv(np.vstack((X[:,:(M-1)], 
                                                      tk[:,:(M-1)])))
        C = CD[:len(YI), :n_states]          # Output matrix
        D = CD[:len(YI), n_states:]          # Feedthrough matrix

        E = yk[:,:(M-1)] - C @ X[:,:(M-1)] - D @ tk[:,:(M-1)]
        ABK = X[:,1:M] @ np.linalg.pinv(np.vstack((X[:,:(M-1)], 
                                                   tk[:,:(M-1)], E)))
        A = ABK[:,:n_states]                 # State transition matrix
        B = ABK[:,n_states:(n_states+N_uy)]  # Input matrix
        K = ABK[:,(n_states+N_uy):]          # Kalman gain
        
        self.ident_time = time() - start     # Time elapsed for identification
        
        self.A, self.B, self.C, self.D, self.K = A, B, C, D, K
        self.X = X
       
    def init_sim(self, Z_test_sc):
        # Calculate initial state x(0) using CVA projection matrix
        tk = self.kpca_uy.transform(Z_test_sc)
        tk_p = np.flip(tk[:self.n_lags, :].reshape(1, -1))
        Yp_scaled = self.sc_p.transform(tk_p)
        x0 = self.Jn @ Yp_scaled.T
        t0 = tk[self.n_lags, :].reshape(-1, 1)
        return x0, t0
    
    def simulate(self, Z_test):
        start = time()
        Nt = Z_test.shape[0]
        Z_test_sc = self.sc_raw.transform(Z_test)
        x_pred = np.zeros((self.n_states, Nt - self.n_lags + 1)) 
        y_pred = np.zeros((len(self.YI), Nt - self.n_lags + 1))
        x0, t0 = self.init_sim(Z_test_sc)
        y0 = self.C @ x0 + self.D @ t0
        x_pred[:, 0] = x0.ravel()
        y_pred[:, 0] = y0.ravel()
        exit_code = 0
        
        for j in np.arange(1, y_pred.shape[1]):
            zk = np.hstack((Z_test_sc[j+self.n_lags-1, self.UI], 
                            y_pred[:, j-1]))
            tk = self.kpca_uy.transform(zk.reshape(1, -1))
            xk_1 = x_pred[:, j-1].reshape(-1, 1)
            xk = self.A @ xk_1 + self.B @ tk.T
            yk = self.C @ xk + self.D @ tk.T
            x_pred[:, j] = xk.ravel()
            y_pred[:, j] = yk.ravel()
            if (np.abs(yk) > 1e3).any():
                exit_code = -1
        
        x_pred = np.transpose(x_pred)
        y_pred = np.transpose(y_pred)
        y_pred = (y_pred * self.sc_raw.scale_[self.YI]) + \
                           self.sc_raw.mean_[self.YI]
        self.sim_time = time() - start
        
        return x_pred, y_pred, exit_code
    
    def R2_score(self, Z_test, y_pred):
        r2 = np.zeros(len(self.YI))
        for k in range(len(self.YI)):
            y_true = Z_test[self.n_lags-1:, self.YI[k]]
            r2[k] = 1 - np.sum((y_true - y_pred[:, k]) ** 2) \
                    / np.sum((y_true - np.mean(y_true)) ** 2)   
        return r2
    
    def display(self):
        print(f'No. of medoids: {self.kpca_uy.N_red}')
        print(f'No. of KPCs on [u y]: {self.kpca_uy.n_comp}')
        print(f'No. of lags: {self.n_lags}')
        print(f'No. of states: {self.n_states}')
        print(f'Indices of u: {self.UI}')
        print(f'Indices of y: {self.YI}')
        print('State-space matrices:')
        print(self.A)
        print(self.B)
        print(self.C)
        print(self.D)
        print(self.K)

## Load all models

In [3]:
cva_mdl = pickle.load(open('evap_cva_sys.pkl','rb'))
mkcva_mdl = pickle.load(open('evap_mkcva_sys3.pkl','rb'))
lstm_mdl = load_model('evap_lstm.keras')

look_back = 15
YI, UI = np.array([3, 4, 5]), np.array([0, 1, 2])
input_spec = tf.TensorSpec([None, look_back, len(UI)+len(YI)], dtype=tf.float32)
lstm_func = tf.function(lstm_mdl).get_concrete_function(input_spec)

# Use Tensorflow's XLA (Accelerated Linear Algebra) for faster inference
@tf.function(jit_compile=True)
def lstm_predict(x):
    return lstm_func(tf.cast(x, tf.float32))

# Standardscaler params of original Training Data (for LSTM use only)
scale_ = np.array([30.52634951, 27.30680801, 3.31043849, 0.12226375, 2.27643356, 4.95861316])
mean_ = np.array([204.82968912, 196.14955658, 49.74290531, 1.0075573, 50.58551774, 25.41497448])

## Methods for evaluating models

In [4]:
def prepare_data(evap_df):
    noise_var = np.array([0, 0, 0, 0.1, 1, 1])
    v_name = ['F200', 'P100', 'F3', 'L2', 'P2', 'X2']
    timepts = np.linspace(0, 1000, 1001)
    res = list()
    for j in range(len(noise_var)):
        data = np.interp(timepts, evap_df.index.values, evap_df[v_name[j]].values)
        data += (np.random.rand(len(data))-0.5)*noise_var[j]
        res.append(data)
    
    return np.transpose(np.vstack(res))

def eval_pred(Z, y_pred):
    r2 = np.zeros(len(YI))
    for k in range(len(YI)):
        y_true = Z[-len(y_pred):, YI[k]]
        r2[k] = 1 - np.sum((y_true - y_pred[:, k]) ** 2) \
                / np.sum((y_true - np.mean(y_true)) ** 2)
    
    return r2
    
def eval_CVA_simulate(Z):
    sim_time = time()
    x_pred, y_pred, exit_code = cva_mdl.simulate(Z)
    sim_time = time() - sim_time
    r2 = eval_pred(Z, y_pred)
    return r2, sim_time

def eval_MKCVA_simulate(Z):
    sim_time = time()
    x_pred, y_pred, exit_code = mkcva_mdl.simulate(Z)
    sim_time = time() - sim_time
    r2 = eval_pred(Z, y_pred)
    return r2, sim_time

def eval_LSTM_simulate(Z):
    start = time()
    uy0 = Z[:look_back, :]
    u = Z[look_back-1:, UI]
    
    K.clear_session()
    
    N = u.shape[0]
    y_pred = np.zeros((N, len(YI)))
    uy = (uy0 - mean_) / scale_
    u_ = (u - mean_[UI]) / scale_[UI]
    y_pred[0, :] = uy[-1, YI]
    
    for j in np.arange(1, N):
        uy = np.hstack((np.vstack((uy[1:, UI], 
                                   u_[np.minimum(j, N), :])), 
                        uy[:, YI]))
        y = lstm_predict(uy[np.newaxis, :, :])
        y_pred[j, :] = y[:, -1, :]
        uy = np.hstack((uy[:, UI],
                        np.vstack((uy[1:, YI], y[:, -1, :]))))
    
    y_pred = y_pred * scale_[YI] + mean_[YI]
    sim_time = time() - start
    r2 = eval_pred(Z, y_pred)
    return r2, sim_time

## Define Pyomo model and simulate

In [5]:
def solve_model(ti, tf, F200data, P100data, F3data, init_data):

    evap = ConcreteModel()

    evap.ti = Param(initialize=ti)
    evap.tf = Param(initialize=tf)
    evap.t = ContinuousSet(bounds=(evap.ti,evap.tf))

    # States
    evap.L2 = Var(evap.t, initialize=1, bounds=(0, 4))
    evap.P2 = Var(evap.t, initialize=50.5)
    evap.X2 = Var(evap.t, initialize=25)

    # Manipulated variable to control L2 via P-control
    evap.F2 = Var(evap.t, initialize=2)
    
    # Inputs and Disturbances
    evap.T200 = Param(evap.t, default=25)
    evap.F1 = Param(evap.t, default=10)
    evap.X1 = Param(evap.t, default=5)
    evap.T1 = Param(evap.t, default=40)

    evap.F200 = Param(evap.t, mutable=True)  # Input to be manipulated
    evap.P100 = Param(evap.t, mutable=True)  # Input to be manipulated
    evap.F3 = Param(evap.t, mutable=True)    # Input to be manipulated

    # Other outputs
    evap.F4 = Var(evap.t, initialize=8)
    evap.F5 = Var(evap.t, initialize=8)
    evap.T2 = Var(evap.t, initialize=84.6)
    evap.T3 = Var(evap.t, initialize=80.6)
    evap.F100 = Var(evap.t, initialize=9.27)
    evap.T100 = Var(evap.t, initialize=119.9)
    evap.Q100 = Var(evap.t, initialize=339.2)
    evap.T201 = Var(evap.t, initialize=46.15)
    evap.Q200 = Var(evap.t, initialize=308)

    # Setup derivative vars for states
    evap.dL2dt = DerivativeVar(evap.L2, initialize=init_data['dL2dt'])
    evap.dP2dt = DerivativeVar(evap.P2, initialize=init_data['dP2dt'])
    evap.dX2dt = DerivativeVar(evap.X2, initialize=init_data['dX2dt'])

    # Set an objective
    evap.obj = Objective(expr=1)

    evap.z1dot = Constraint(evap.t, rule = lambda m, i: \
                           m.dL2dt[i]*20 == m.F1[i] - m.F4[i] - m.F2[i])
    evap.z2dot = Constraint(evap.t, rule = lambda m, i: \
                           m.dX2dt[i]*20 == m.F1[i]*m.X1[i] - m.F2[i]*m.X2[i])
    evap.z3dot = Constraint(evap.t, rule = lambda m, i: \
                           m.dP2dt[i]*4 == m.F4[i] - m.F5[i])

    # Other constraints
    evap.con1 = Constraint(evap.t, rule = lambda m, i: \
                          m.T2[i] == 0.5616*m.P2[i] + 0.3126*m.X2[i] + 48.43)
    evap.con2 = Constraint(evap.t, rule = lambda m, i: \
                          m.T3[i] == 0.507*m.P2[i] + 55)
    evap.con3 = Constraint(evap.t, rule = lambda m, i: \
                          m.F4[i]*38.5 == m.Q100[i] - 0.07*m.F1[i]*(m.T2[i] - m.T1[i]))
    evap.con4 = Constraint(evap.t, rule = lambda m, i: \
                          m.T100[i] == 0.1538*m.P100[i] + 90)
    evap.con5 = Constraint(evap.t, rule = lambda m, i: \
                          m.Q100[i] == 0.16*(m.F1[i] + m.F3[i])*(m.T100[i] - m.T2[i]))
    evap.con6 = Constraint(evap.t, rule = lambda m, i: \
                          m.F100[i]*36.6 == m.Q100[i])
    evap.con7 = Constraint(evap.t, rule = lambda m, i: \
                          m.Q200[i]*(0.14*m.F200[i]+6.84) == 0.9576*m.F200[i]*(m.T3[i]-m.T200[i]))
    evap.con8 = Constraint(evap.t, rule = lambda m, i:\
                          m.T201[i] == m.T200[i] + m.Q200[i]/0.07/m.F200[i])
    evap.con9 = Constraint(evap.t, rule = lambda m, i: \
                          m.F5[i]*38.5 == m.Q200[i])

    def _init(m):
        yield m.L2[evap.ti] == init_data['L2']
        yield m.P2[evap.ti] == init_data['P2']
        yield m.X2[evap.ti] == init_data['X2']
    
    evap.initcon = ConstraintList(rule=_init)

    # Discretize using collocation
    discretizer = TransformationFactory('dae.collocation')
    discretizer.apply_to(evap, nfe=25, ncp=3, scheme='LAGRANGE-RADAU')

    # P-controller
    def _p_control(m, i):
        if i > m.ti:
            return m.F2[i] == 2 + 5*(m.L2[m.t.prev(i)] - 1.0)
        else:
            return m.F2[i] == init_data['F2']

    evap.p_control = Constraint(evap.t, rule=_p_control)
    
    # Step change data
    timepoints = list(evap.t)
    if evap.ti.value == 0:
        F200data[0] = 208
        P100data[0] = 194.7
        F3data[0] = 50
    for i, t in enumerate(timepoints):
        pos = np.argwhere(t>=i_data)[-1]
        evap.F200[t] = F200data[pos][0]
        evap.P100[t] = P100data[pos][0]
        evap.F3[t] = F3data[pos][0]

    # Solve using Pyomo IPOPT
    solver = SolverFactory('cyipopt')
    res = solver.solve(evap)
    
    model_vars = evap.component_map(ctype=Var)
    model_params = evap.component_map(ctype=Param)

    s_list = []
    col_list = []
    ctr = 1
    for k in model_vars.keys():
        v = model_vars[k]
        s = pd.Series(v.extract_values(), 
                      index=v.extract_values().keys())
        s.sort_index(inplace=True)
        s_list.append(s)
        col_list.append(v.name)
        ctr += 1

    for k in model_params.keys():
        v = model_params[k]
        if v.name == 'F200' or v.name == 'P100' or v.name == 'F3':
            s = pd.Series(v.extract_values(), 
                          index=v.extract_values().keys())
            s.sort_index(inplace=True)
            s_list.append(s)
            col_list.append(v.name)
            ctr += 1

    evap_df = pd.concat(s_list, axis=1)
    evap_df.columns = col_list
    return evap_df, res

#rand_seed = 102
#np.random.seed(rand_seed)
i_data = np.arange(0, 1000, 50) # start, last, increment

cva_r2, cva_time = [], []
mkcva_r2, mkcva_time = [], []
lstm_r2, lstm_time = [], []

for trial in range(50):

    # For extrapolation data (50% larger)
    #F200data = (np.random.rand(i_data.shape[0])-0.5)*150 + 208
    #P100data = (np.random.rand(i_data.shape[0])-0.5)*150 + 194.7
    #F3data = (np.random.rand(i_data.shape[0])-0.5)*15 + 50

    # For extrapolation data (25% larger)
    #F200data = (np.random.rand(i_data.shape[0])-0.5)*125 + 208
    #P100data = (np.random.rand(i_data.shape[0])-0.5)*125 + 194.7
    #F3data = (np.random.rand(i_data.shape[0])-0.5)*12.5 + 50

    # For interpolation data
    F200data = (np.random.rand(i_data.shape[0])-0.5)*100 + 208
    P100data = (np.random.rand(i_data.shape[0])-0.5)*100 + 194.7
    F3data = (np.random.rand(i_data.shape[0])-0.5)*10 + 50

    init_data = {'L2':1,    'P2':50.5, 'X2':25, 'F2':2,
                 'dL2dt':0, 'dP2dt':0, 'dX2dt':0}

    # Initialize the data frame
    evap_df = pd.DataFrame(columns=['L2', 'P2', 'X2', 'dL2dt', 'dP2dt', 
                                    'dX2dt', 'F200', 'P100', 'F3'])

    # Set the horizon length and total time
    hor_len = 25
    total_time = 1000
    overall_status = True
    sim_time = time()

    for j in np.arange(0, total_time, hor_len): # start, last, increment

        # Simulate the model at time [j, j+hor_len]
        temp_df, res = solve_model(j, j+hor_len, F200data, P100data, F3data, init_data)
        overall_status = overall_status and (res.Solver.status == 'ok')

        # Save the last condition as the next initial condition
        for k in init_data.keys():
            init_data[k] = temp_df.iloc[-1][k]

        # Append temp_df after evap_df
        if j+hor_len < total_time:
            evap_df = pd.concat([evap_df, temp_df.iloc[:-1,:]], axis=0)
        else:
            evap_df = pd.concat([evap_df, temp_df], axis=0)

    sim_time = time() - sim_time
    print(f'Trial {trial}: [{sim_time:.2f} sec], OK? {overall_status}')
    
    Z = prepare_data(evap_df)
    
    r2, sim_time = eval_CVA_simulate(Z)
    cva_r2.append(np.mean(r2))
    cva_time.append(sim_time)
    
    r2, sim_time = eval_MKCVA_simulate(Z)
    mkcva_r2.append(np.mean(r2))
    mkcva_time.append(sim_time)
    
    r2, sim_time = eval_LSTM_simulate(Z)
    lstm_r2.append(np.mean(r2))
    lstm_time.append(sim_time)

df = pd.DataFrame(np.column_stack([cva_r2, cva_time, mkcva_r2, mkcva_time, lstm_r2, lstm_time]),
                  columns=['CVA-R2', 'CVA-Time', 'MKCVA-R2', 
                           'MKCVA-Time', 'LSTM-R2', 'LSTM-Time'])
print(df.head())
df.to_csv('compare_sysID.csv', index=False) 

Please recompile / update your pynumero_ASL library.
Trial 0: [8.05 sec], OK? True
Trial 1: [7.75 sec], OK? True
Trial 2: [7.80 sec], OK? True
Trial 3: [7.82 sec], OK? True
Trial 4: [7.83 sec], OK? True
Trial 5: [7.67 sec], OK? True
Trial 6: [7.81 sec], OK? True
Trial 7: [8.18 sec], OK? True
Trial 8: [7.69 sec], OK? True
Trial 9: [7.88 sec], OK? True
Trial 10: [7.87 sec], OK? True
Trial 11: [8.06 sec], OK? True
Trial 12: [7.76 sec], OK? True
Trial 13: [7.87 sec], OK? True
Trial 14: [7.91 sec], OK? True
Trial 15: [7.71 sec], OK? True
Trial 16: [7.74 sec], OK? True
Trial 17: [7.77 sec], OK? True
Trial 18: [8.05 sec], OK? True
Trial 19: [7.78 sec], OK? True
Trial 20: [7.89 sec], OK? True
Trial 21: [7.87 sec], OK? True
Trial 22: [7.91 sec], OK? True
Trial 23: [7.79 sec], OK? True
Trial 24: [7.82 sec], OK? True
Trial 25: [8.26 sec], OK? True
Trial 26: [7.87 sec], OK? True
Trial 27: [7.79 sec], OK? True
Trial 28: [7.85 sec], OK? True
Trial 29: [7.84 sec], OK? True
Trial 30: [7.68 sec], OK? T